In [14]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Definimos URL Base
urlbase = 'https://www.enisa.es/es/comunidad-enisa/prestamos/searchresult?pag='

i=1

# Iteramos para las primeras 10 páginas de resultado de búsqueda - por tanto los 120 préstamos más recientes
while i<11:
    # Construimos la URL
    url = urlbase + str(i)
    #print(url)
    try:
     page = urlopen(url)
    except:
     print('Error opening the URL')

    # Creamos el objeto bs para la URL
    soup = BeautifulSoup(page,'html.parser', from_encoding='utf-8')

    # Ponemos foco en el area que nos interesa
    t_row_header = soup.find_all('th')
    t_row_data = soup.find_all('td')
    
    # Condicion de salida
    if (len(t_row_data)==0): break
    
    header = []
    data = {}
    
    # En la primera iteración leemos cabecera y creamos dataframe
    if i==1:
        for row in t_row_header:
            h = str.strip(row.p.text)
            header.append(h)

        df_enisa = pd.DataFrame(columns=header)
    
    # Empezamos a iterar para todas las filas con contenido de la página
    # Guardamos la info en el dataframe
    for row in t_row_data:
        col=str.strip(row.b.text)
        value=str.strip(row.div.find(text=True, recursive=False))
        data[col]=value
        if col=='CC.AA.': 
            df_enisa=df_enisa.append(data, ignore_index=True)
            data = {}

    i=i+1

# Creamos el CSV con los datos
df_enisa.to_csv('df_enisa.csv', sep=";", na_rep= "",)
